In [1]:
from ctypes import *
import numpy as np
import time

In [2]:
import os
os.getcwd()

'/home/dyu/Dropbox/tf-notebooks/cuda_comp/fista_example/using_dll'

In [3]:
libc_FISTA_dll = CDLL("./fista_kernel.so")
FISTA_Kernel_D = libc_FISTA_dll.FISTA_D

In [4]:
libc_FISTA_blas = CDLL("./fista_cublas.so")
FISTA_blas_D = libc_FISTA_blas.FISTA_D

In [50]:
FISTA_Kernel_D.restype = None
FISTA_Kernel_D.argtypes = (POINTER(c_double), POINTER(c_double), POINTER(c_double), 
                         POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

In [51]:
FISTA_blas_D.restype = None
FISTA_blas_D.argtypes = (POINTER(c_double), POINTER(c_double), POINTER(c_double), 
                         POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

In [52]:
n = 500
p = 1000

import numpy as np
np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)

In [53]:
## input
beta = np.zeros(p).astype(np.float64)
beta_cr = beta.ctypes.data_as(POINTER(c_double))
X_cr =  X.ctypes.data_as(POINTER(c_double))## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
y_cr = y.ctypes.data_as(POINTER(c_double))

In [54]:
## Parameters
loss = np.zeros(10000).astype(np.float64)## max_iter size만큼
loss_c = loss.ctypes.data_as(POINTER(c_double)) 
lam =  np.sqrt(2 * np.log(p) / n)
lam_c = c_double(lam)
L_c = c_double(10)
eta_c = c_double(2)
tol_c = c_double(1e-04)

iter_c = c_int(10000)
n_c = c_int(n)
p_c = c_int(p)
step_c = c_int(0)

In [55]:
tic = time.time()
FISTA_Kernel_D(beta_cr, X_cr, y_cr, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
toc = time.time()
print("Elapsed time: ", toc - tic, "secs")

Elapsed time:  2.5550453662872314 secs


In [56]:
step_c

c_int(1624)

In [28]:
loss

array([3.25126311e+00, 1.04598456e+00, 7.23532077e-01, ...,
       2.84848637e-04, 2.72734073e-04, 2.66540958e-04])

In [62]:
## input
beta = np.zeros(p).astype(np.float64)
beta_cc = beta.ctypes.data_as(POINTER(c_double))
X_c =  X.ravel(order = "F")              ## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
X_cc = X_c.ctypes.data_as(POINTER(c_double))
#y_c = y.ravel(order = "F")
y_cc = y.ctypes.data_as(POINTER(c_double))

loss = np.zeros(10000).astype(np.float64)## max_iter size만큼
loss_c = loss.ctypes.data_as(POINTER(c_double)) 
lam =  np.sqrt(2 * np.log(p) / n)
lam_c = c_double(lam)
L_c = c_double(10)
eta_c = c_double(2.0)
tol_c = c_double(1e-04)

iter_c = c_int(5000)
n_c = c_int(n)
p_c = c_int(p)
step_c = c_int(0)

In [63]:
tic = time.time()
FISTA_blas_D(beta_cc, X_cc, y_cc, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
toc = time.time()
    
print("Elapsed time: ", toc - tic, "secs")

Elapsed time:  1.1587193012237549 secs


In [64]:
step_c

c_int(1550)

In [18]:
beta[:10]

array([0.98551965, 0.9991859 , 0.97275275, 0.9404534 , 0.96161795,
       0.93524647, 0.9418953 , 1.0280432 , 0.8438578 , 0.92526376],
      dtype=float32)

In [1]:
n = 100
p = 200

import time
import numpy as np
from ctypes import *

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)

libc_FISTA_dll = CDLL("./fista_kernel.so")
FISTA_Kernel_D = libc_FISTA_dll.FISTA_D

FISTA_Kernel_D.restype = None
FISTA_Kernel_D.argtypes = (POINTER(c_double), POINTER(c_double), POINTER(c_double), 
                         POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

libc_FISTA_dll2 = CDLL("./fista_dll_v2.so")
FISTA_Kernel_D2 = libc_FISTA_dll.FISTA_D

FISTA_Kernel_D2.restype = None
FISTA_Kernel_D2.argtypes = (POINTER(c_double), POINTER(c_double), POINTER(c_double), 
                         POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))


niter = 11
comp_time_ker = np.zeros(niter)
comp_time_ker2 = np.zeros(niter)


for i in np.arange(niter):
    
    beta = np.zeros(p).astype(np.float64)
    beta_cr = beta.ctypes.data_as(POINTER(c_double))
    X_cr =  X.ctypes.data_as(POINTER(c_double))## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
    y_cr = y.ctypes.data_as(POINTER(c_double))


    loss = np.zeros(10000).astype(np.float64)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_double)) 
    lam =  np.sqrt(2 * np.log(p) / n)
    lam_c = c_double(lam)
    L_c = c_double(10)
    eta_c = c_double(2.0)
    tol_c = c_double(1e-04)

    iter_c = c_int(10000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_Kernel_D(beta_cr, X_cr, y_cr, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_ker[i] = t2-t1
    
    beta = np.zeros(p).astype(np.float64)
    beta_cr = beta.ctypes.data_as(POINTER(c_double))
    X_cr =  X.ctypes.data_as(POINTER(c_double))## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
    y_cr = y.ctypes.data_as(POINTER(c_double))


    loss = np.zeros(10000).astype(np.float64)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_double)) 
    lam =  np.sqrt(2 * np.log(p) / n)
    lam_c = c_double(lam)
    L_c = c_double(10)
    eta_c = c_double(2.0)
    tol_c = c_double(1e-04)

    iter_c = c_int(10000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_Kernel_D2(beta_cr, X_cr, y_cr, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_ker2[i] = t2-t1
    
    
    print('\n ',i+1,'-th iteration, Collapsed Time-1: ', comp_time_ker[i], ', Collapsed Time-2S: ', comp_time_ker2[i])
    



  1 -th iteration, Collapsed Time-1:  0.44771265983581543 , Collapsed Time-2S:  0.2726149559020996

  2 -th iteration, Collapsed Time-1:  0.2708168029785156 , Collapsed Time-2S:  0.27063608169555664

  3 -th iteration, Collapsed Time-1:  0.2704792022705078 , Collapsed Time-2S:  0.27022361755371094

  4 -th iteration, Collapsed Time-1:  0.27050304412841797 , Collapsed Time-2S:  0.27005839347839355

  5 -th iteration, Collapsed Time-1:  0.27028560638427734 , Collapsed Time-2S:  0.27100610733032227

  6 -th iteration, Collapsed Time-1:  0.26993417739868164 , Collapsed Time-2S:  0.26970362663269043

  7 -th iteration, Collapsed Time-1:  0.27100324630737305 , Collapsed Time-2S:  0.27085375785827637

  8 -th iteration, Collapsed Time-1:  0.2713906764984131 , Collapsed Time-2S:  0.2704732418060303

  9 -th iteration, Collapsed Time-1:  0.27069091796875 , Collapsed Time-2S:  0.27082109451293945

  10 -th iteration, Collapsed Time-1:  0.27007365226745605 , Collapsed Time-2S:  0.270494699478149

In [2]:
n = 500
p = 1000

import time
import numpy as np
from ctypes import *

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)

libc_FISTA_blas = CDLL("./fista_cublas.so")
FISTA_blas_D = libc_FISTA_blas.FISTA_S

FISTA_blas_D.restype = None
FISTA_blas_D.argtypes = (POINTER(c_double), POINTER(c_double), POINTER(c_double), 
                         POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

niter = 11
comp_time_blas = np.zeros(niter)


for i in np.arange(niter):
    
    beta = np.zeros(p).astype(np.float64)
    beta_cc = beta.ctypes.data_as(POINTER(c_double))
    X_c =  X.ravel(order = "F")              ## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
    X_cc = X_c.ctypes.data_as(POINTER(c_double))
    #y_c = y.ravel(order = "F")
    y_cc = y.ctypes.data_as(POINTER(c_double))

    loss = np.zeros(10000).astype(np.float64)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_double)) 
    lam =  np.sqrt(2 * np.log(p) / n)
    lam_c = c_double(lam)
    L_c = c_double(10)
    eta_c = c_double(2.0)
    tol_c = c_double(1e-04)

    iter_c = c_int(5000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_blas_D(beta_cc, X_cc, y_cc, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_blas[i] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_blas[i])
    



  1 -th iteration, Collapsed Time:  0.8475148677825928

  2 -th iteration, Collapsed Time:  0.5502657890319824

  3 -th iteration, Collapsed Time:  0.5511596202850342

  4 -th iteration, Collapsed Time:  0.5509366989135742

  5 -th iteration, Collapsed Time:  0.5512492656707764

  6 -th iteration, Collapsed Time:  0.5491976737976074

  7 -th iteration, Collapsed Time:  0.5507485866546631

  8 -th iteration, Collapsed Time:  0.5501794815063477

  9 -th iteration, Collapsed Time:  0.5496270656585693

  10 -th iteration, Collapsed Time:  0.5495586395263672

  11 -th iteration, Collapsed Time:  0.5509135723114014
